In [1]:
import pandas as pd
import numpy as np
import helper
import project_helper
import project_tests

In [2]:
df = pd.read_csv('eod-quotemedia.csv')
df.head(10)
print(df.shape)
print(df.columns)

(490737, 6)
Index(['Unnamed: 0', 'date', 'ticker', 'adj_close', 'adj_volume', 'dividends'], dtype='object')


In [5]:
percent_top_dollar = 0.2
high_volume_symbols = project_helper.large_dollar_volume_stocks(df, 'adj_close', 'adj_volume', percent_top_dollar)
df = df[df['ticker'].isin(high_volume_symbols)]

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')
volume = df.reset_index().pivot(index='date', columns='ticker', values='adj_volume')
dividends = df.reset_index().pivot(index='date', columns='ticker', values='dividends')
close.head(20)

ticker,AAPL,AMZN,FB
date,,,
2013-07-01,53.10917319,282.10000000,24.81000000
2013-07-02,54.31224742,283.73000000,24.41000000
2013-07-03,54.61204262,284.03000000,24.52000000
2013-07-05,54.17338125,285.88000000,24.37000000
2013-07-08,53.86579916,290.59000000,24.71000000
2013-07-09,54.81320389,291.53000000,25.48000000
2013-07-10,54.60295791,292.33000000,25.80000000
2013-07-11,55.45406479,299.66000000,25.81000000
2013-07-12,55.35309481,307.55000000,25.91000000


In [6]:
project_helper.print_dataframe(close)

In [13]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    #TODO: Implement function
    # create dollar_volume
    dollar_volume = close * volume
    
    # Normalization
    dollar_vol_weights = dollar_volume.divide(dollar_volume.sum(axis=1), axis='index')

    return dollar_vol_weights

project_tests.test_generate_dollar_volume_weights(generate_dollar_volume_weights)

Tests Passed


In [27]:
etf_weights = calculate_dividend_weights(dividends)
project_helper.plot_weights(etf_weights, 'ETF Weights')

ticker           AAPL       AMZN         FB
date                                       
2013-07-01 0.00000000 0.00000000 0.00000000
2013-07-02 0.00000000 0.00000000 0.00000000
2013-07-03 0.00000000 0.00000000 0.00000000
2013-07-05 0.00000000 0.00000000 0.00000000
2013-07-08 0.00000000 0.00000000 0.00000000
ticker           AAPL       AMZN         FB
date                                       
2013-07-01 0.00000000 0.00000000 0.00000000
2013-07-02 0.00000000 0.00000000 0.00000000
2013-07-03 0.00000000 0.00000000 0.00000000
2013-07-05 0.00000000 0.00000000 0.00000000
2013-07-08 0.00000000 0.00000000 0.00000000


In [16]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    #TODO: Implement function
    print(dividends.head())
    
    # cumsum along column
    cum_sum_dividend = dividends.cumsum(axis=0)
    print(cum_sum_dividend.head())
    
    # normalize by dividing by cum_sum_dividend's row sum
    dividend_weights = cum_sum_dividend.divide(cum_sum_dividend.sum(axis=1), axis='index')
    
    return dividend_weights

project_tests.test_calculate_dividend_weights(calculate_dividend_weights)

                  QOI       MKDD       KZVF
2006-09-25 0.00000000 0.00000000 0.00000000
2006-09-26 0.00000000 0.00000000 0.10000000
2006-09-27 0.00000000 1.00000000 0.30000000
2006-09-28 0.00000000 0.20000000 0.00000000
                  QOI       MKDD       KZVF
2006-09-25 0.00000000 0.00000000 0.00000000
2006-09-26 0.00000000 0.00000000 0.10000000
2006-09-27 0.00000000 1.00000000 0.40000000
2006-09-28 0.00000000 1.20000000 0.40000000
Tests Passed


In [20]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """
    #TODO: Implement function
    
    # returns = Pt/Pt-1 -1
    returns = prices/prices.shift(1) -1

    return returns

print(generate_returns(close).shape)
project_tests.test_generate_returns(generate_returns)

(1009, 3)
Tests Passed


In [18]:
returns = generate_returns(close)
project_helper.plot_returns(returns, 'Close Returns')

In [21]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)
    
    #TODO: Implement function
    
    # simply multiply returns and weights for computing weighted returns
    
    weighted_returns = returns * weights

    return weighted_returns

project_tests.test_generate_weighted_returns(generate_weighted_returns)

Tests Passed


In [36]:


# Import pandas library 
import pandas as pd 
  
# initialize list elements 
data = {'apple':[1,3,5,7], 'bana':[2,3,5,7]}
  
# Create the pandas DataFrame with column name is provided explicitly 
df3 = pd.DataFrame(data) 
res = df3.cumsum(axis=0)
print(res)

   apple  bana
0      1     2
1      4     5
2      9    10
3     16    17


In [28]:
index_weighted_returns = generate_weighted_returns(returns, index_weights)
etf_weighted_returns = generate_weighted_returns(returns, etf_weights)
project_helper.plot_returns(index_weighted_returns, 'Index Returns')
project_helper.plot_returns(etf_weighted_returns, 'ETF Returns')